In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="blue-book", port="auto", dev_mode=True, reset=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

## Add Containers

In [ ]:
result = domain_client.api.services.container.add_image(name="python", tag="python:3.10-slim")
result

In [ ]:
azure_cli_dockerfile = """
FROM python:3.10-slim as build

RUN DEBIAN_FRONTEND=noninteractive \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    python3-dev make build-essential git

RUN pip install -U pip
RUN pip install azure-cli==2.50.0
"""

In [ ]:
result = domain_client.api.services.container.add_image(
    name="azure",
    tag="azure-cli:latest",
    dockerfile=azure_cli_dockerfile
)
result

In [ ]:
result = domain_client.api.services.container.get_images()
result

In [ ]:
result[1].tag, result[1].dockerfile

## Build a Container

In [ ]:
result = domain_client.api.services.container.build_image(name="doesntexist")
result

In [ ]:
result = domain_client.api.services.container.build_image(name="azure")
result

## Create a Container Command

In [ ]:
# example command:
# az storage blob upload --file ~/Downloads/nick.jpeg --blob-url https://20230720testblob.blob.core.windows.net/test/nick.jpg
# \?sp\=rw\&st\=2023-07-19T23:15:18Z\&se\=2023-09-01T07:15:18Z\&spr\=https\&sv\=2022-11-02\&sr\=c\ --overwrite

In [ ]:
upload = sy.ContainerUpload(command_file="file", sandbox_path="/sandbox")

In [ ]:
file = sy.ContainerCommandKwarg(name="file", equals=" ", value=upload, required=True)

In [ ]:
inputs = {"uploaded_file": sy.ContainerUpload(command_file="file")}

In [ ]:
sig_secret = "\&sig\=" # to keep out of github

In [ ]:
blob_url = sy.ContainerCommandKwargTemplate(
    name="blob-url",
    value="https://20230720testblob.blob.core.windows.net/test/{uploaded_file}",
    inputs=inputs,
    append="\?sp\=rw\&st\=2023-07-19T23:15:18Z\&se\=2023-09-01T07:15:18Z\&spr\=https\&sv\=2022-11-02\&sr\=c" + sig_secret,
    equals=" ",
)

In [ ]:
overwrite_kwarg = sy.ContainerCommandKwargBool(name="overwrite", value=True, flag=True)

In [ ]:
kwargs = {
    "file": file,
    "blob-url": blob_url,
    "overwrite": overwrite_kwarg
}

In [ ]:
command = sy.ContainerCommand(
    module_name="blue_book",
    name="azure_upload",
    image_name="azure",
    command="az",
    args="storage blob upload",
    kwargs=kwargs,
    user_kwargs=["file", "overwrite"]
)

In [ ]:
command

In [ ]:
orca = sy.SyftFile.from_path("~/Downloads/orca.png")

In [ ]:
command.cmd(run_files={"file": orca})

In [ ]:
result = domain_client.api.services.container.add_command(command=command)
result

In [ ]:
result = domain_client.api.services.container.get_commands()
result

In [ ]:
orca = sy.SyftFile.from_path("~/Downloads/orca.png")

In [ ]:
orca

In [ ]:
result = domain_client.api.services.blue_book.azure_upload(file=orca)
result

In [ ]:
print(result.exit_code)
if len(result.jsonstd):
    print(result.jsonstd[0])

In [ ]:
# Cleanup local domain server
node.land()